In [4]:
import csv
import cv2
import numpy as np


In [5]:
def open_csv(filename) :
    lines = []
    with open(filename) as csvfile :
        reader = csv.reader(csvfile)
        for line in reader :
            lines.append(line)
    return lines
    

In [6]:
lines = open_csv("./data/data/driving_log.csv")

In [ ]:
images = []
measurements = []
corrections = [0,0.2,-0.2]
for line in lines[1:] :
    for i in range(0,3) :
        source_path = line[i]
        correction  = corrections[i]
        filename = source_path.split('/')[-1]
        current_path = './data/data/IMG/' + filename
        image = cv2.imread(current_path)
        images.append(image)
        measurement = float(line[3]) + correction
        measurements.append(measurement)

In [ ]:
lines = open_csv("./data/train_data/driving_log.csv")

In [ ]:
for line in lines[1:] :
    for i in range(0,3) :
        source_path = line[i]
        correction  = corrections[i]
        filename = source_path.split('/')[-1]
        current_path = './data/train_data/IMG/' + filename
        image = cv2.imread(current_path)
        images.append(image)
        measurement = float(line[3]) + correction
        measurements.append(measurement)

In [ ]:
augmented_images = []
augmented_measurements = []

for image,measurement in zip(images,measurements) :
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image,1))
    augmented_measurements.append(measurement * -1.0)


In [7]:
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(lines[1:], test_size=0.2)
print(len(validation_samples))

1608


In [16]:
from sklearn.utils import shuffle
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        #print("Entered")
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            
            images = []
            measurements = []
            corrections = [0,0.2,-0.2]
            for line in batch_samples :
                for i in range(0,3) :
                    source_path = line[i]
                    correction  = corrections[i]
                    filename = source_path.split('/')[-1]
                    current_path = './data/data/IMG/' + filename
                    #print(current_path + '\n')
                    image = cv2.imread(current_path)
                    images.append(image)
                    measurement = float(line[3]) + correction
                    measurements.append(measurement)
                    images.append(cv2.flip(image,1))
                    measurement = float(line[3]) + correction
                    measurements.append(measurement * -1.0)
                    
            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(measurements)
            #print(X_train.shape)
            yield shuffle(X_train, y_train)

In [17]:
# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

In [52]:
test = generator(train_samples, batch_size=32)
print(train_generator.shape)

AttributeError: 'generator' object has no attribute 'shape'

In [39]:
#print(len(augmented_images))
X_train = np.array(augmented_images[0:23000])
y_train = np.array(augmented_measurements[0:23000])
print(X_train.shape)

NameError: name 'augmented_images' is not defined

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Flatten,Lambda,Cropping2D
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

In [20]:

model = Sequential()
model.add(Lambda(lambda x : x / 255.0,input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Convolution2D(6,5,5,activation='relu'))
model.add(MaxPooling2D())
model.add(Convolution2D(6,5,5,activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))
model.compile(loss='mse',optimizer='adam')
#model.fit(X_train,y_train,validation_split=0.2,shuffle=True,nb_epoch=6)
model.fit_generator(train_generator, samples_per_epoch=
            len(train_samples)* 4, validation_data=validation_generator,
            nb_val_samples=len(validation_samples), nb_epoch=1)

Epoch 1/1
576/642 [=========================>....] - ETA: 1s - loss: 4.8306

/Users/girishpai/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1574: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


768/642 [===================================] - 31s - loss: 4.2867 - val_loss: 3.8780


In [ ]:
model.save('model.h5')